# Pokemon scraper

Gebruikte libraries

In [2]:
import requests
import math
import time
import pandas as pd
from bs4 import BeautifulSoup

### PokemonDB name scraper 

In [94]:
def scrapeDBName(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)

    pokemon_name = soup.find("h1").get_text()
    #Ethiek
    time.sleep(2)  
    return {"PokemonName": pokemon_name}

In [3]:
scrapeDBName(1)

{'PokemonName': 'Bulbasaur'}

In [4]:
sentence = "dragon flying"
splitted = sentence.split()


### Pokemon Type function

In [95]:
def scrapeDBType(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    #POKEMON TYPE
    row_counter = 0

    #er zijn meerdere tables met deze class, maar omdat de data die we nodig hebben enkel in de 1ste is gebruiken we gewoon soup.find()
    pokemon_type_table = soup.find("table",{"class": "vitals-table"})

    for table_row in pokemon_type_table.find_all("tr"):
        row_counter += 1
        
        if row_counter == 2:
            pokemon_type = table_row.find("td").get_text().strip()

            if(" " in pokemon_type != True):                               
                type_list= []
                type_list = pokemon_type.split()               
                Type1 = type_list[0]
                Type2 = type_list[1]
                return {"Type1": Type1, "Type2" : Type2}
 
            else:            
                return {"Type1": pokemon_type, "Type2" : "None"} 
    
    time.sleep(2)

In [6]:
scrapeDBType("rayquaza")

{'Type1': 'Dragon', 'Type2': 'Flying'}

### Pokemon stats function

In [81]:
def scrapeDBStats(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    
    #POKEMON STATS
    table_counter = 0
    row_counter = 0
    
    pokemonName_stats = []

    #er zijn meerdere tables met dezelfde class en geen unieke ID dus gaan we alle tables met deze class moeten scrapen
    all_stat_tables = soup.find_all("table",{"class": "vitals-table"})
    #loop om juiste table te vinden
    for stat_table in all_stat_tables:
        table_counter += 1
        
        #we  willen enkel de 4de table scrapen
        if table_counter == 4:
            
            #alle rijen in table loopen en table dimensions in rij vinden
            for table_row in stat_table.find_all("tr"):
                #print("row found")
                row_counter += 1
                #row_counter is nodig om "total" niet mee te nemen
                if row_counter <= 6:
                    dimensions = table_row.find_all("td")
                    header = table_row.find("th")
                    print(header.get_text() + ": " + dimensions[0].get_text())    
    #Ethiek
    time.sleep(3)  

### Pokemon Moves function

In [82]:
def scrapeDBMoves(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    #POKEMON MOVES
    #LEVEL MOVES
    ms_moves = []

    levelup_moves_table = soup.find("table",{"class" : "data-table"})

    body_levelup_moves = levelup_moves_table.find("tbody")

    for move_row in body_levelup_moves.find_all("td",{"class":"cell-name"}):
        ms_moves.append(move_row.get_text())

        #EGG MOVES
    table_counter = 0    
    tables = soup.find_all("table",{"class" : "data-table"})

    for table in tables:
        table_counter += 1
        
        if table_counter == 2:
            body_egg_moves = table.find("tbody")
            for move_row in body_egg_moves.find_all("td",{"class":"cell-name"}):
                ms_moves.append(move_row.get_text())
    
    #EGG MOVES
    table_counter = 0    
    tables = soup.find_all("table",{"class" : "data-table"})

    for table in tables:
        table_counter += 1
        
        if table_counter == 3:
            body_egg_moves = table.find("tbody")
            for move_row in body_egg_moves.find_all("td",{"class":"cell-name"}):
                ms_moves.append(move_row.get_text())
    
    #TM MOVES
    table_counter = 0    
    tables = soup.find_all("table",{"class" : "data-table"})

    for table in tables:
        table_counter += 1
        
        if table_counter == 4:
            body_tm_moves = table.find("tbody")
            for move_row in body_tm_moves.find_all("td",{"class":"cell-name"}):
                ms_moves.append(move_row.get_text())
    
    print(ms_moves,"\n", len(ms_moves))    



In [9]:
scrapeDBName(384)
#scrapeDBType(pokemonID)
#scrapeDBStats(pokemonID)
#scrapeDBMoves(pokemonID)

{'PokemonName': 'Rayquaza'}

### Converting stats

The scraped stats are not the final stats of a Pokemon. As pogo is a more simplified game, the stats for each pokemon are condensed to 3 instead of 6. 

Interessant: haal aan in het uitschrijven dat we perfecte IV's toevoegen aan elke stat

In [83]:
def statConversion(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    
    #POKEMON STATS
    table_counter = 0
    row_counter = 0
    
    pokemonName_stats = []

    all_stat_tables = soup.find_all("table",{"class": "vitals-table"})
    
    #print("\nPOKEMON MS STATS \n")

    for stat_table in all_stat_tables:
        table_counter += 1
        
        if table_counter == 4:
            
            for table_row in stat_table.find_all("tr"):                
                row_counter += 1
                
                if row_counter <= 6:
                    dimensions = table_row.find_all("td")
                    header = table_row.find("th")
                    #print(dimensions[0].get_text())
                    pokemonName_stats.append(dimensions[0].get_text())

    #print("\nPOKEMON GO STATS \n")

    #Speedmod
    speedMod = 1 + (int(pokemonName_stats[5])-75)/500

    #Attack
    scaledAttack = round(2*((7/8)*max(int(pokemonName_stats[1]),int(pokemonName_stats[3])) + (1/8)*min(int(pokemonName_stats[1]),float(pokemonName_stats[3]))))
    baseAttack = round(scaledAttack*speedMod) 
    baseAttackandIV = baseAttack + 15
    #print("Attack:",baseAttack)

    #Defence
    scaledDefense = round(2*((5/8)*max(int(pokemonName_stats[2]),int(pokemonName_stats[4]))+(3/8)*min(int(pokemonName_stats[2]),int(pokemonName_stats[4]))))
    baseDefense = round(scaledDefense*speedMod)
    baseDefenseandIV = baseDefense +15
    #print("Defense:",baseDefense)  

    #HP
    baseStamina = math.floor(int(pokemonName_stats[0]) * 1.75 + 50)
    baseStaminaandIV = baseStamina + 15
    #print("Stamina:",math.ceil(baseStamina),"\n") 

    #CP
    cplvl_40 = math.floor(max(10,(baseAttackandIV*(math.pow(baseDefenseandIV,0.5))*(math.pow(baseStaminaandIV,0.5))*(math.pow(0.7903,2)))/10))
    #print("before nerf:", cplvl_40)    
    if cplvl_40 >= 4000:
        baseStamina = math.ceil(baseStamina * 0.91) +15
        baseAttack = math.ceil(baseAttack * 0.91) +15
        baseDefense = round(baseDefense * 0.91) +15        
        cplvl_40 = round(max(10,((baseAttack)*(math.pow((baseDefense),0.5))*(math.pow((baseStamina),0.5))*(math.pow(0.7903,2)))/10))
        #print("after nerf:", cplvl_40)
        #print("\n",baseAttack - 15, baseDefense - 15, baseStamina - 15)
        return {"HP" : int(baseStamina)-15,"Attack": int(baseAttack),"Defense" : int(baseDefense)-15,"CP": int(cplvl_40)}
    else:
        return {"HP" : int(baseStamina),"Attack": int(baseAttack),"Defense" : int(baseDefense),"CP": int(cplvl_40)}
    
  

In [11]:
statConversion(629)


{'HP': 172, 'Attack': 105, 'Defense': 138, 'CP': 1267}

### PokemonGO moves

Hier gaan we te werk met data van https://gamepress.gg/pokemongo/ 

In [84]:
def gamepressScraperMoves(pokemonID):
    url = "https://gamepress.gg/pokemongo/pokemon/"+str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)    
    fastAttack = "no fast attack"
    chargedAttack = "no charged attack"

    try:
        table = soup.find("table",{"class":"moveset-table"})
        tableBody = table.find("tbody")

        rows = tableBody.find_all("tr")

        if not rows:
            fastAttack = "no fast attack"
            chargedAttack = "no charged attack"
            return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}
        
        for row in tableBody.find_all("tr"):
            columns = row.find_all("td")

            if columns[2].get_text() == "A":            
                fastAttack = columns[0].get_text().strip()
                chargedAttack = columns[1].get_text().strip()
                return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}                
            else:
                fastAttack = "no fast attack"
                chargedAttack = "no charged attack"
                return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}                
                
    except:
        fastAttack = "no fast attack"
        chargedAttack = "no charged attack"
        return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}

In [92]:
def gamepressScraperMoves1(pokemonID):
    url = "https://gamepress.gg/pokemongo/pokemon/"+str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)    
    fastAttack = "no fast attack"
    chargedAttack = "no charged attack"

    try:
        table = soup.find("table",{"class":"moveset-table"})
        tableBody = table.find("tbody")
        rows = tableBody.find_all("tr")

        if not rows:
            fastAttack = "no fast attack"
            chargedAttack = "no charged attack"
            return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}
        
        for row in rows:
            columns = row.find_all("td")

            if columns[2].get_text().strip() == "A":            
                fastAttack = columns[0].get_text().strip()
                chargedAttack = columns[1].get_text().strip()
                return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}   
                         
            else:
                fastAttack = "no fast attack"
                chargedAttack = "no charged attack"
                return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}                
                
    except:
        fastAttack = "no fast attack"
        chargedAttack = "no charged attack"
        return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}

In [77]:
gamepressScraperMoves1(6)





Fire Spin









Overheat





A-
X


{'Fast Attack': 'no fast attack', 'Charged Attack': 'no charged attack'}

### Pokemon Ranking

In [85]:
def gamepressScraperRanking(pokemonID):
    url = "https://gamepress.gg/pokemongo/pokemon/"+str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)

    try:
        all_ranks = ["S","A+","A","B+","B","C",]
        ranking = soup.find("span",{"class":"cell-rating-right"}).get_text()
        ranking = ranking.replace(" Tier","")        

        if ranking not in all_ranks:
            ranking = "F"
            return{"Rank": ranking}
        
        else:
            return{"Rank": ranking}

    except:
        ranking = "F"
        return{"Rank": ranking}

In [39]:
gamepressScraperRanking(384)

{'Rank': 'A+'}

In [16]:
for i in range(0,3) :
    print(scrapeDBType(1).values())

dict_values(['Grass', 'Poison'])
dict_values(['Grass', 'Poison'])
dict_values(['Grass', 'Poison'])


In [34]:
cols = ["PokemonName", "Type1","Type2","HP","Attack","Defense","CP","Fast Attack","Charged Attack","Rank"]
df = pd.DataFrame( columns=cols)
for i in range(1,5):
    data1 = scrapeDBName(i)
    data2 = scrapeDBType(i)
    data3 = statConversion(i)
    data4 = gamepressScraperMoves(i)    
    time.sleep(1)
    df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))
df

<ipython-input-34-c957c3054680>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))
<ipython-input-34-c957c3054680>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))
<ipython-input-34-c957c3054680>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))
<ipython-input-34-c957c3054680>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))


,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,Charged Attack
0,Bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,128.0,118.0,111.0,1115.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no fast attack,no charged attack
0,Ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,155.0,151.0,143.0,1699.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no fast attack,no charged attack
0,Venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
pokemon_data_list = []

for pokemonID in range(1,10):  # Change the range as needed
    data1 = scrapeDBName(pokemonID)
    data2 = scrapeDBType(pokemonID)
    data3 = statConversion(pokemonID)
    data4 = gamepressScraperMoves(pokemonID)
    data5 = gamepressScraperRanking(pokemonID)
    time.sleep(1)
    # Merge alle dictionaries 
    merged_data = {**data1, **data2, **data3, **data4, **data5}    
    pokemon_data_list.append(merged_data)

df = pd.DataFrame(pokemon_data_list)

print(df)

  PokemonName  Type1   Type2   HP  Attack  Defense    CP     Fast Attack  \
0   Bulbasaur  Grass  Poison  128     118      111  1115  no fast attack   
1     Ivysaur  Grass  Poison  155     151      143  1699  no fast attack   
2    Venusaur  Grass  Poison  190     198      189  2720  no fast attack   
3  Charmander   Fire    None  118     116       93   980  no fast attack   
4  Charmeleon   Fire    None  151     158      126  1653  no fast attack   
5   Charizard   Fire  Flying  186     223      173  2889  no fast attack   
6    Squirtle  Water    None  127      94      121   946  no fast attack   
7   Wartortle  Water    None  153     126      155  1488  no fast attack   
8   Blastoise  Water    None  188     171      207  2466  no fast attack   
9    Caterpie    Bug    None  128      55       55   437  no fast attack   

      Charged Attack Rank  
0  no charged attack    F  
1  no charged attack    F  
2  no charged attack    B  
3  no charged attack    F  
4  no charged attack   

In [67]:
df

,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,Charged Attack,Rank
0,Bulbasaur,Grass,Poison,128,118,111,1115,no fast attack,no charged attack,F
1,Ivysaur,Grass,Poison,155,151,143,1699,no fast attack,no charged attack,F
2,Venusaur,Grass,Poison,190,198,189,2720,no fast attack,no charged attack,B
3,Charmander,Fire,None,118,116,93,980,no fast attack,no charged attack,F
4,Charmeleon,Fire,None,151,158,126,1653,no fast attack,no charged attack,F
5,Charizard,Fire,Flying,186,223,173,2889,no fast attack,no charged attack,C
6,Squirtle,Water,None,127,94,121,946,no fast attack,no charged attack,F
7,Wartortle,Water,None,153,126,155,1488,no fast attack,no charged attack,F
8,Blastoise,Water,None,188,171,207,2466,no fast attack,no charged attack,F
9,Caterpie,Bug,None,128,55,55,437,no fast attack,no charged attack,F


In [44]:

IDS = []

In [75]:
url = "https://serebii.net/pokemongo/gen9pokemon.shtml"
request = requests.get(url)
soup = BeautifulSoup(request.text)

table = soup.find_all("table",{"class":"tab"})
tableBody = table[1]

for row in tableBody.find_all("tr"):
    dimension = row.find_all("td",{"class": "fooinfo"})

    if len(dimension) == 3:        
        name = dimension[0].get_text()

        if "Not Currently Available" not in name:
            number = row.find("td").get_text().strip()
            number = number.replace('#','')
            
            if len(number) == 4:
                if number[0] == "0":                    
                    number = number[1:]
                    if number[0] == "0":
                        number = number[1:]
                        if number[0] == "0":
                            number = number[1:]

            IDS.append(number)
            
for i in range(0,len(IDS)):
    IDS[i] = int(IDS[i])
IDS = list(set(IDS))

IDS.sort()
for item in IDS:
    print(item)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [97]:
pokemon_data_list = []
for i in range(0,len(IDS)):
    data1 = scrapeDBName(IDS[i])
    data2 = scrapeDBType(IDS[i])
    data3 = statConversion(IDS[i])
    data4 = gamepressScraperMoves1(IDS[i])
    data5 = gamepressScraperRanking(IDS[i])
    time.sleep(1)
    # Merge alle dictionaries 
    merged_data = {**data1, **data2, **data3, **data4, **data5}    
    pokemon_data_list.append(merged_data)

df = pd.DataFrame(pokemon_data_list)

In [98]:
df

,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,Charged Attack,Rank
0,Bulbasaur,Grass,Poison,128,118,111,1115,no fast attack,no charged attack,F
1,Ivysaur,Grass,Poison,155,151,143,1699,no fast attack,no charged attack,F
2,Venusaur,Grass,Poison,190,198,189,2720,no fast attack,no charged attack,B
3,Charmander,Fire,None,118,116,93,980,no fast attack,no charged attack,F
4,Charmeleon,Fire,None,151,158,126,1653,no fast attack,no charged attack,F
...,...,...,...,...,...,...,...,...,...,...
838,Frigibax,Dragon,Ice,163,134,86,1247,no fast attack,no charged attack,F
839,Arctibax,Dragon,Ice,207,173,128,2092,no fast attack,no charged attack,F
840,Baxcalibur,Dragon,Ice,229,269,167,3541,Ice Fang,Avalanche,B+
841,Gimmighoul,Ghost,None,128,121,122,1188,no fast attack,no charged attack,F


In [ ]:
df.to_csv("pokemon_dataset.csv")